In [0]:
!pip install Faker

In [0]:
# from pyspark.sql import functions as F
# from faker import Faker
# import random
# from collections import OrderedDict 
# from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
# import uuid

In [0]:
# # folder = "s3://caden-os-dev-databricks-tmp/ling"
# folder = "s3://caden-os-dev-demo-staging/ling"
# # dbutils.fs.rm(folder, True)
# dbutils.fs.ls(folder)

In [0]:
# fake = Faker()

# @udf(StructType([StructField("id", StringType(), False),
#                 StructField("firstname", StringType(), False),
#                 StructField("lastname", StringType(), False),
#                 StructField("email", StringType(), False),
#                 StructField("address", StringType(), False),
#                 StructField("operation", StringType(), False),
#                 StructField("operation_date", StringType(), False)]))
# def generate_fake_data():
#     return (
#         str(uuid.uuid4()),
#         fake.first_name(),
#         fake.last_name(),
#         fake.ascii_company_email(),
#         fake.address(),
#         random.choice(["APPEND", "DELETE", "UPDATE"]),
#         fake.date_time_this_month().strftime("%m-%d-%Y %H:%M:%S")
#     )

# df = spark.range(0, 100000)
# df = df.withColumn("fake_data", generate_fake_data())
# df = df.selectExpr("fake_data.id", "fake_data.firstname", "fake_data.lastname", "fake_data.email", "fake_data.address", "fake_data.operation", "fake_data.operation_date")
# df.repartition(100).write.format("json").mode("overwrite").save(folder+"/customers")

In [0]:
# spark.read.json(folder+"/customers").display()

In [0]:
# import dlt
# from pyspark.sql.functions import *
# from pyspark.sql.types import *

In [0]:
# # source = spark.conf.get("source")
# source = folder

In [0]:
# @dlt.table(name="customer_bronze",
#                   comment = "New customer data incrementally ingested from cloud object storage landing zone",
#   table_properties={
#     "quality": "bronze"
#   }
# )
# def customer_bronze():
#   return (
#     spark.readStream.format("cloudFiles") \
#       .option("cloudFiles.format", "json") \
#       .option("cloudFiles.inferColumnTypes", "true") \
#       .load(f"{source}/customers")
#   )

In [0]:
# @dlt.table(name="customer_bronze_clean_v",
#   comment="Cleansed bronze customer view (i.e. what will become Silver)")

# @dlt.expect_or_drop("valid_id", "id IS NOT NULL")
# @dlt.expect("valid_address", "address IS NOT NULL")
# @dlt.expect_or_drop("valid_operation", "operation IS NOT NULL")
# def customer_bronze_clean_v():
#   return (dlt.read_stream("customer_bronze")
#             .select("address", "email", "id", "firstname", "lastname", "operation", "operation_date", "_rescued_data")
#          )

In [0]:
# dlt.create_target_table(name="customer_silver",
#   comment="Clean, merged customers",
#   table_properties={
#     "quality": "silver"
#   }
# )
# dlt.apply_changes(
#   target = "customer_silver", #The customer table being materilized
#   source = "customer_bronze_clean_v", #the incoming CDC
#   keys = ["id"], #Primary key to match the rows to upsert/delete
#   sequence_by = col("operation_date"), #deduplicate by operation date getting the most recent value
#   apply_as_deletes = expr("operation = 'DELETE'"), #DELETE condition
#   except_column_list = ["operation", "operation_date", "_rescued_data"], # drop metadata columns
#   stored_as_scd_type = 2
# )

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
json_path = "/databricks-datasets/wikipedia-datasets/data-001/clickstream/raw-uncompressed-json/2015_2_clickstream.json"
@dlt.create_table(
  comment="The raw wikipedia clickstream dataset, ingested from /databricks-datasets."
)
def clickstream_raw():          
  return (
    spark.read.json(json_path)
  )

In [0]:
# @dlt.table(
#   comment="Wikipedia clickstream data cleaned and prepared for analysis."
# )
# @dlt.expect("valid_current_page_title", "current_page_title IS NOT NULL")
# @dlt.expect_or_fail("valid_count", "click_count > 0")
# def clickstream_prepared():
#   return (
#     dlt.read("clickstream_raw")
#       .withColumn("click_count", expr("CAST(n AS INT)"))
#       .withColumnRenamed("curr_title", "current_page_title")
#       .withColumnRenamed("prev_title", "previous_page_title")
#       .select("current_page_title", "click_count", "previous_page_title")
#   )

In [0]:
# @dlt.table(
#   comment="A table containing the top pages linking to the Apache Spark page."
# )
# def top_spark_referrers():
#   return (
#     dlt.read("clickstream_prepared")
#       .filter(expr("current_page_title == 'Apache_Spark'"))
#       .withColumnRenamed("previous_page_title", "referrer")
#       .sort(desc("click_count"))
#       .select("referrer", "click_count")
#       .limit(10)
#   )
